In [148]:
import sympy as sp
import numpy as np
sp.init_printing(use_unicode=True)
np.set_printoptions(suppress=True)

In [149]:
delta_x = 1/4
delta_y = 1/4

x_min = 0
x_max = 1
x_target = 1/2

y_min = 0
y_max = 1
y_target = 1/2

boundary_conditions = {
    'top_side': 'u',
    'right_side': 'du',
    'bottom_side': 'u',
    'left_side': 'u',
}

if 'u' in boundary_conditions.values():
    if 'du' in boundary_conditions.values():
        conditions = 'cauchy'
    else:
        conditions = 'dirichlet'
else:
    conditions = 'neumann'

def u(x, y):
    # The order in which these boundary conditions are defined is important!
    if x == x_min:
        return 1
    if y == y_min:
        return 0  
    if y == y_max:
        return 1
    if x == x_max:
        return 0
    
def du(x, y):
    if x == x_min:
        return 0
    if y == y_min:
        return 0
    if x == x_max:
        return 0
    if y == y_max:
        return 0
    
forcing_term = 0

n = int(np.round((x_max-x_min)/delta_x)) + 1
m = int(np.round((y_max-y_min)/delta_y)) + 1
if boundary_conditions['top_side'] == 'du':
    n += 1
if boundary_conditions['bottom_side'] == 'du':
    n += 1
if boundary_conditions['right_side'] == 'du': 
    m += 1
if boundary_conditions['left_side'] == 'du':
    m += 1
dims = (n, m)

In [150]:
def numgrid(dims):
    output = np.zeros((dims))
    n, m = dims
    c = 1
    for i in range(1, m-1):
        for j in range(1, n-1):
            output[j, i] = c
            c += 1
    return output

In [151]:
G = numgrid(dims)

In [155]:
def delsq(G, conditions=None):
    n, m = G.shape
    output_shape = ((n-2)*(m-2), (n-2)*(m-2))
    output = np.zeros(output_shape)
    b = np.zeros((output_shape[0], 1))
    row = 0
    for i in range(1, m-1):
        for j in range(1, n-1):
            up = int(G[j-1][i])
            right = int(G[j][i+1])
            down = int(G[j+1][i])
            left = int(G[j][i-1])
            
#             print(f"{up} {right} {down} {left}")
         
            if up:
                output[row][up-1] = -1
            else:
#                 print("Up")
#                 print(f"i = {j}, j = {i}")
                if boundary_conditions['top_side'] == 'u':
                    val = u(i*delta_x, y_max)
                else:
                    val = -2*delta_x*du(i*delta_x, y_max)
                b[row][0] += val
#                 print(f"Added: {val}")
                    
#                 print("\n")
                
            if right:
                output[row][right-1] = -1
            else:
#                 print("Right")
#                 print(f"i = {j}, j = {i}")
                if boundary_conditions['right_side'] == 'u':
                    val = u(x_max, (n-1-j)*delta_y)
                else:
                    val = -2*delta_x*du(x_max, (n-1-j)*delta_y)
                b[row][0] += val
#                     print(f"Added: {val}")
                     
#                 print("\n")

            if down:
                output[row][down-1] = -1
            else:
#                 print("Down")
#                 print(f"i = {j}, j = {i}")
                if boundary_conditions['bottom_side'] == 'u':
                    val = u(i*delta_x, y_min)
                else:
                    val = -2*delta_x*du(i*delta_x, y_min)
                b[row][0] += val
#                     print(f"Added: {val}")
                    
#                 print("\n")
                
            if left:
                output[row][left-1] = -1
            else:
#                 print("Left")
#                 print(f"i = {j}, j = {i}")
                if boundary_conditions['bottom_side'] == 'u':
                    val = u(x_min, (n-1-j)*delta_y)
                else:
                    val = -2*delta_x*du(x_min, (n-1-j)*delta_y)
                b[row][0] += val
#                     print(f"Added: {val}")
                             
#                 print("\n")
                
            output[row][row] = 4
    
#             print("----------------")

            row += 1
    
    if conditions == 'cauchy':
        if boundary_conditions['top_side'] == 'du':
            # This requires a different mesh
            pass
        
        if boundary_conditions['right_side'] == 'du':
            for i in range(1, n-1):
                col = int(G[:,n-2][1:n-1][i-1] - 1)
                row = int(G[:,n-1][1:n-1][i-1] - 1)
                output[row][col] = -2
                
        if boundary_conditions['bottom_side'] == 'du':
            # This requires a different mesh
            pass
        
        if boundary_conditions['left_side'] == 'du':
            for i in range(1, n-1):
                col = int(G[:,2][1:n-1][i-1] - 1)
                row = int(G[:,1][1:n-1][i-1] - 1)
                output[row][col] = -2
                
        for i in range(1, n-1):
            row = int(G[:,n-1][1:n-1][i-1] - 1)
            b[row]
                
    return output, b

In [156]:
A, b = delsq(G, conditions=conditions)

In [157]:
vals = np.dot(np.linalg.inv(A), b)
sp.Matrix(np.round(vals, decimals=4)).T

[0.9008  0.7684  0.5348  0.8348  0.6379  0.3709  0.8007  0.5774  0.311  0.7904
  0.5602  0.2955]